In [1]:
import matplotlib.pyplot as plt
import uproot
import numpy as np
import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split

from datetime import datetime
import sys

In [2]:
now = datetime.now()
print("time at start =", now)

time at start = 2021-09-24 21:04:33.352174


In [3]:
HTCondorRun = str(sys.argv[1])
print("HTCondorRun:",HTCondorRun)

HTCondorRun: -f


In [4]:
save_data = True
tmp_data = False
take_subset = True
subset_size = 100000

In [5]:
nfs_path = "/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/"

data_subdir = "Dstlnu_skim_BsBtag_separation_{}_evts/".format(subset_size)   
root_subdir = "axheim_DstlnuSKIM_run1/"   

root_path = nfs_path + "Dstlnu_skim/" + root_subdir

In [6]:
merged = "merged_"
if tmp_data:
    merged += "tmp_"

In [7]:
fileY4S = uproot.open(root_path + merged + "DXtagDstl.root:variables")

In [8]:
names = ["gammas","electrons","pions","kaons","muons"]
dfs = []
for name in names:
    filename = root_path + merged + "{}.root:variables".format(name)
    print(filename)
    tmpFileFSPs = uproot.open(filename)
    df_tmp = tmpFileFSPs.arrays(library="pd")
    dfs.append(df_tmp)

/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_gammas.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_electrons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_pions.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_kaons.root:variables
/nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/Dstlnu_skim/axheim_DstlnuSKIM_run1/merged_muons.root:variables


In [9]:
df_FSPs = pd.concat(dfs)

In [10]:
df_Y4S = fileY4S.arrays(library="pd")

In [11]:
print(df_FSPs.shape[0])


12556867


In [12]:
# delete FSPs for which no Y4S file entry was found
#df_FSPs = df_FSPs[df_FSPs['__event__'].isin(df_Y4S["__event__"])]

In [13]:
print(df_FSPs.shape[0])


12556867


## take a sample if used in notebook for faster processing

In [14]:
all_evt_nums = df_FSPs['__event__'].unique()
all_evt_nums.shape[0]

465660

In [15]:
sample_evt_nums = np.random.choice(all_evt_nums, size=subset_size)
sample_evt_nums.shape[0]

100000

In [16]:
if take_subset:
    df_FSPssample = df_FSPs[df_FSPs['__event__'].isin(sample_evt_nums)]
    #df_Y4S=df_Y4Ssample
    df_FSPs=df_FSPssample

In [17]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("numEvents:",df_FSPs['__event__'].unique().shape[0])

df_FSPs.shape[0]: 2430333
numEvents: 90074


### delete particles which occur more than ones based on uniqueParticleIdentifier

In [18]:
groupsFSPs_uniqParID = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","uniqueParticleIdentifier"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsFSPs_uniqParID.sort_values("count"))

In [19]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])
print("groupsFSPs_uniqParID.shape[0]:",groupsFSPs_uniqParID.shape[0])
print("df_Y4S.shape[0]:",df_Y4S.shape[0])

df_FSPs.shape[0]: 2430333
groupsFSPs_uniqParID.shape[0]: 1663236
df_Y4S.shape[0]: 11771


In [20]:
# delete particles which occur more than ones (keep first) and if possible keep the one with basf2_used==1
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())
df_FSPs = df_FSPs.sort_values("basf2_used",ascending=False).drop_duplicates(subset=("__event__","uniqueParticleIdentifier"), keep='first')
print("df_FSPs[basf2_used].value_counts():",df_FSPs["basf2_used"].value_counts())

df_FSPs[basf2_used].value_counts(): 0.0    2391884
1.0      38449
Name: basf2_used, dtype: int64
df_FSPs[basf2_used].value_counts(): 0.0    1624787
1.0      38449
Name: basf2_used, dtype: int64


In [21]:
print("df_FSPs.shape[0]:",df_FSPs.shape[0])

df_FSPs.shape[0]: 1663236


## save sample 

In [22]:
if take_subset:
    df_FSPs.to_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S.to_csv(root_path + "df_Y4S_sample__evts.csv")

In [23]:
load = False
if load:
    df_FSPs = pd.read_csv(root_path + "df_FSPs_sample_{}_evts.csv".format(subset_size))
    #df_Y4S = pd.read_csv(root_path + "df_Y4S_sample10evts.csv")

### print one event

In [24]:
df_FSPs["__event__"]

147008     5931037
1010701    9667434
1010696    9667434
3137522    3411839
1010697    9667434
            ...   
4230488    6297615
4230489    6297615
4230490    6297615
4230491    6297615
4230492    6297615
Name: __event__, Length: 1663236, dtype: int32

In [25]:
df_FSPs[df_FSPs["__event__"] == 6620998].sort_values('mcPDG')[['mcPDG','genMothPDG_0', 'mcMother0_uniqParID', 'genMotherID_0',
'genMothPDG_1', 'mcMother1_uniqParID', 'genMotherID_1',
'genMothPDG_2', 'mcMother2_uniqParID', 'genMotherID_2',
'genMothPDG_3', 'mcMother3_uniqParID', 'genMotherID_3',
'genMothPDG_4', 'mcMother4_uniqParID', 'genMotherID_4',
'genMothPDG_5', 'mcMother5_uniqParID', 'genMotherID_5',
'genMothPDG_6', 'mcMother6_uniqParID', 'genMotherID_6',
'genMothPDG_7', 'mcMother7_uniqParID', 'genMotherID_7',
'genMothPDG_8', 'mcMother8_uniqParID', 'genMotherID_8',
'genMothPDG_9', 'mcMother9_uniqParID', 'genMotherID_9']]

,mcPDG,genMothPDG_0,mcMother0_uniqParID,genMotherID_0,genMothPDG_1,mcMother1_uniqParID,genMotherID_1,genMothPDG_2,mcMother2_uniqParID,genMotherID_2,...,genMotherID_6,genMothPDG_7,mcMother7_uniqParID,genMotherID_7,genMothPDG_8,mcMother8_uniqParID,genMotherID_8,genMothPDG_9,mcMother9_uniqParID,genMotherID_9


## check if category combinations make sense

In [26]:
groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
groupsAllFSPs

,basf2_used,basf2_Bsig,basf2_X,count
0,0.0,0.0,0.0,1624787
1,1.0,0.0,0.0,8465
2,1.0,0.0,1.0,18158
3,1.0,1.0,0.0,11826


### add B_ID col with extra info

In [27]:
# function to create col with the particles mother B's uniqueParticleIdentifier
def B_ID(s):
    label = 0
    for i in range(10):
        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
        if ((s[mcMotheri_uniqParID]) == 83886082.0):
            label = 83886082   
        elif ((s[mcMotheri_uniqParID]) == 83886081.0):
            label = 83886081   
    return label
df_FSPs['B_ID'] = df_FSPs.apply(B_ID, axis=1)

In [28]:
# function to create col with info which i-th daughter of B the particle is (0=daughter,1=granddaughter,.. -1=no B daughter)
#def B_iDaughter(s):
#    label = -1
#    for i in range(10):
#        mcMotheri_uniqParID = "mcMother{}_uniqParID".format(i)
#        if ((s[mcMotheri_uniqParID] == 83886082.0) or (s[mcMotheri_uniqParID] == 83886081.0)):
#            label = i   
#            return label
#    return label
#df_FSPs['B_iDaughter'] = df_FSPs.apply(B_iDaughter, axis=1)

In [29]:
#df_FSPs["mcPDG_abs"] = df_FSPs["mcPDG"].abs()
#mcPDG_BID_combis = pd.DataFrame({'count' : df_FSPs.groupby( ["mcPDG_abs","B_ID","B_iDaughter"] ).size()}).reset_index()


#printDF = mcPDG_BID_combis[mcPDG_BID_combis["B_iDaughter"]==0].sort_values('B_ID',ascending=False)

#pd.set_option('display.float_format', lambda x: '%.0f' % x)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(printDF)

In [30]:
# this shows that Hc is sometimes combined from both B's which is of course wrong
#groupsAllFSPs = pd.DataFrame({'count' : df_FSPs.groupby( ["__event__","B_ID","Hc","basf2_used","basf2_Bsig","basf2_X"] ).size()}).reset_index()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(groupsAllFSPs.sort_values("Hc"))

## data preprocessing

### save dataframes on NFS

In [31]:
if HTCondorRun == "isHTCondorRun":
    df_FSPs.to_csv(root_path + "df_FSPs_preProcessed_{}_evts.csv".format(subset_size))
    

### load dataframes

In [32]:
#df_FSPs = pd.read_csv(root_path + "df_FSPs_preProcessed.csv")
#event_Bs = pd.read_csv(root_path + "event_Bs.csv")

In [33]:
df_FSPs_final = df_FSPs

# start of NN data creation

In [34]:
numFSPs = pd.DataFrame({'count' : df_FSPs_final.groupby( ["__event__"] ).size()}).reset_index()

minFSPs = numFSPs["count"].min()
maxFSPs = numFSPs["count"].max()
print("minFSPs:",minFSPs)
print("maxFSPs:",maxFSPs,'\n')

df_FSPs_final['numFSPs'] = df_FSPs_final.groupby('__event__')['__event__'].transform('count')

minFSPs: 4
maxFSPs: 41 



In [35]:
data_dir = Path(nfs_path + "data/" + data_subdir + root_subdir)    
if save_data:
    data_dir.mkdir(parents=True, exist_ok=True)
print("Will save data to:", data_dir,'is', save_data ,'\n')

Will save data to: /nfs/dust/belle2/user/axelheim/MC_studies/Dstlnu_Bt_generic/data/Dstlnu_skim_BsBtag_separation_100000_evts/axheim_DstlnuSKIM_run1 is True 



In [36]:
for num_FSPs_toData in range(minFSPs, maxFSPs+1):
    df_num_subset = df_FSPs_final.copy()
    df_num_subset = df_num_subset[df_num_subset['numFSPs'] == num_FSPs_toData]
    
        
    numEvents = df_num_subset.__event__.nunique()
    print("numEvents:",numEvents)
    print("num_FSPs_toData:",num_FSPs_toData)  
    if numEvents == 0:
        print("skipped because empty \n")
        continue

    if numEvents < 10:
        print("skipped because <10 events \n")
        continue
    
    num_features = 4
    leaves = np.zeros((numEvents, num_FSPs_toData,  num_features))  
    SA_target =  np.zeros((numEvents, num_FSPs_toData))
    global_tag = np.chararray((numEvents, num_FSPs_toData + 1), itemsize=30)
    
    event_list = df_num_subset[df_num_subset["numFSPs"] == num_FSPs_toData]["__event__"].unique()
    #print("len(event_list):",len(event_list))
    for i in range(numEvents):

        event_iter = event_list[i]

        global_tag_masterInfo = "evt" + str(event_iter)
        global_tag[i,-1] = global_tag_masterInfo
        #print("global_tag[i,-1]:",global_tag[i,-1])
        #print("i:",i,"event_iter:",event_iter)

        event_df = df_num_subset[df_num_subset.__event__ == event_iter]

        for j in range(num_FSPs_toData):
            #print("numParticle:",j)
            particle = event_df.iloc[j]

            #print(particle["mcPDG"],particle["px"],particle["py"],particle["pz"],particle["E"])
            leaves[i,j,0] = particle["px"]
            leaves[i,j,1] = particle["py"]
            leaves[i,j,2] = particle["pz"]
            leaves[i,j,3] = particle["E"]
            
            basf2_usage = "basf2_NONE"
            if particle["basf2_Bsig"] == 1.0:
                basf2_usage = "basf2_Bsig"
            elif particle["basf2_X"] == 1.0:
                basf2_usage = "basf2_X"
            elif particle["basf2_used"] == 0:
                basf2_usage = "basf2_bg"

            global_tag_Info = str((particle["mcPDG"])) 
            global_tag_Info += "_" + basf2_usage
            global_tag[i,j] = global_tag_Info

            label = -10 # error code if assignment fails
            B_tag_uniqID = 83886081.0
            B_sig_uniqID = 83886082.0
            if particle["B_ID"] == B_tag_uniqID:
                label = 1 # particle belongs to Btag (MC truth)
            elif particle["B_ID"] == B_sig_uniqID:
                label = 2 # particle belongs to Bsig (MC truth)
            elif particle["B_ID"] == 0:
                label = 0 # background
            
            
            SA_target[i,j] = label
            
        del event_df
        
        
    # shuffle the data    
    for idx in np.arange(leaves.shape[0]):   # arange is like range but gives ndarray instead of list
        perms = np.random.permutation(leaves.shape[1])

        leaves[idx,:] = leaves[idx,perms]
        SA_target[idx,:] = SA_target[idx,perms]
        global_tag[idx,0:-1] = global_tag[idx,perms]
        
        
         


    #print(global_tag)
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    print("leaves.shape:",leaves.shape)
    print("SA_target.shape:",SA_target.shape)
    print("global_tag.shape:",global_tag.shape)


    print("leaves[0]:",leaves[0])
    print("SA_target[0]:",SA_target[0])
    print("global_tag[0]:",global_tag[0])

    x=leaves
    y=SA_target
    z=global_tag

    x_train, x_test, y_train, y_test, z_train, z_test = train_test_split(x, y, z, test_size=1 - train_ratio, shuffle=False)
    x_val, x_test, y_val, y_test, z_val, z_test = train_test_split(x_test, y_test, z_test, test_size=test_ratio/(test_ratio + validation_ratio), shuffle=False) 

    if save_data==True:
        np.save(data_dir / "leaves_train_FSP{}.npy".format(num_FSPs_toData), x_train)
        np.save(data_dir / "is_left_arr_train_FSP{}.npy".format(num_FSPs_toData), y_train)
        np.save(data_dir / "global_tag_train_FSP{}.npy".format(num_FSPs_toData), z_train)

        np.save(data_dir / "leaves_val_FSP{}.npy".format(num_FSPs_toData), x_val)
        np.save(data_dir / "is_left_arr_val_FSP{}.npy".format(num_FSPs_toData), y_val)
        np.save(data_dir / "global_tag_val_FSP{}.npy".format(num_FSPs_toData), z_val)

        np.save(data_dir / "leaves_test_FSP{}.npy".format(num_FSPs_toData), x_test)
        np.save(data_dir / "is_left_arr_test_FSP{}.npy".format(num_FSPs_toData), y_test)
        np.save(data_dir / "global_tag_test_FSP{}.npy".format(num_FSPs_toData), z_test)

    
    print("")
    #del df_num_subset


    del df_num_subset
                                          

numEvents: 1
num_FSPs_toData: 4
skipped because <10 events 

numEvents: 14
num_FSPs_toData: 5
leaves.shape: (14, 5, 4)
SA_target.shape: (14, 5)
global_tag.shape: (14, 6)
leaves[0]: [[-0.73185778  0.31768751  0.05581185  0.81187192]
 [-0.04516443 -0.76168793 -0.26011336  0.81813641]
 [ 0.0610194  -0.67847484  0.70642853  0.99124802]
 [ 0.88328403  1.44276547  0.17697856  1.70662363]
 [ 0.07529382  0.2502816  -0.05735037  0.30179297]]
SA_target[0]: [2. 1. 2. 2. 1.]
global_tag[0]: [b'211.0_basf2_bg' b'211.0_basf2_bg' b'13.0_basf2_bg' b'-321.0_basf2_bg'
 b'13.0_basf2_bg' b'evt6385243']

numEvents: 59
num_FSPs_toData: 6
leaves.shape: (59, 6, 4)
SA_target.shape: (59, 6)
global_tag.shape: (59, 7)
leaves[0]: [[ 0.0735662  -0.01050718  0.02379765  0.07803021]
 [-0.36740878 -0.08972073 -0.52551091  0.6623297 ]
 [-0.83105797 -0.44158253 -0.36527783  1.01909777]
 [ 0.96426129 -0.34605873  0.58377236  1.18736117]
 [-0.05903549 -0.07231112 -0.02909808  0.09777927]
 [-1.10382962  0.51533413  2.302027


numEvents: 6146
num_FSPs_toData: 15
leaves.shape: (6146, 15, 4)
SA_target.shape: (6146, 15)
global_tag.shape: (6146, 16)
leaves[0]: [[ 0.01191922  0.39113641  0.15884776  0.42232969]
 [ 0.08350334  0.04328512  0.0914937   0.13121549]
 [ 0.04579649  0.02198978  0.06562597  0.08299178]
 [-1.06372344  0.79019815  1.0776546   1.77791369]
 [ 0.10343411  0.17596342 -0.04277414  0.20854584]
 [ 0.1437283   0.05619502 -0.04741166  0.16144216]
 [-0.03425683  0.03512032  0.06577512  0.16190499]
 [ 1.09834909  0.52285987  1.37810194  1.84347445]
 [-0.95554042 -0.56087083 -0.33241361  1.1651662 ]
 [ 0.14523838  0.1396811  -0.05148238  0.53569805]
 [ 0.77619356 -0.19097692 -0.84228265  1.16599845]
 [-0.05184975 -0.10572322 -0.09659047  0.20658027]
 [-0.53889412 -0.2946963   0.09753294  0.63737379]
 [ 0.3597039   0.19581735 -0.35448191  0.55934662]
 [ 0.33214831 -0.04986272  0.46711141  0.59201499]]
SA_target[0]: [1. 0. 1. 2. 1. 1. 2. 1. 1. 1. 2. 1. 1. 2. 1.]
global_tag[0]: [b'22.0_basf2_X' b'nan_ba


numEvents: 5829
num_FSPs_toData: 21
leaves.shape: (5829, 21, 4)
SA_target.shape: (5829, 21)
global_tag.shape: (5829, 22)
leaves[0]: [[-7.43786385e-03  4.39942479e-02  7.90089890e-02  1.66472606e-01]
 [-4.79398817e-01  5.14662325e-01  7.85538405e-02  7.07722573e-01]
 [ 5.73961020e-01 -9.26505923e-01  1.36481357e+00  1.74977834e+00]
 [ 5.69077879e-02 -1.30200773e-01  6.58452660e-02  1.56608866e-01]
 [ 6.88057439e-03 -3.30034969e-03  2.61998232e-02  2.72885571e-02]
 [ 7.66403824e-02  6.21342212e-02  7.25957304e-02  1.22493060e-01]
 [-1.16174281e-01 -8.10507592e-03  1.23060504e-02  1.17105058e-01]
 [-2.42722556e-02 -1.79086924e-02 -8.26780393e-04  3.01752750e-02]
 [-1.15278892e-01 -1.02396905e-01  1.44656032e-01  2.11423076e-01]
 [ 2.49854356e-01 -2.69435734e-01  1.25801697e-01  6.28144770e-01]
 [-2.33374555e-02  6.09699078e-02  2.54166275e-02  7.00569157e-02]
 [-6.45450596e-03 -1.78016275e-02  1.05456458e-02  2.16741606e-02]
 [ 2.38376364e-01  1.39745072e-01  3.06154788e-02  5.66574138e-


numEvents: 2228
num_FSPs_toData: 26
leaves.shape: (2228, 26, 4)
SA_target.shape: (2228, 26)
global_tag.shape: (2228, 27)
leaves[0]: [[ 1.12314336e-01  1.13814445e-02  2.41192859e-02  1.15437374e-01]
 [-1.02141932e-01 -3.02051217e-03  1.12429656e-01  2.06306611e-01]
 [-1.21411577e-01  6.53852522e-02 -3.84580344e-02  1.43160828e-01]
 [ 1.80204865e-02  2.14167032e-02 -2.46433653e-02  3.72922051e-02]
 [-2.82593071e-01  1.11668789e+00  3.28167319e-01  1.20237605e+00]
 [ 6.99842721e-02 -3.22277367e-01  2.51268268e-01  4.14603717e-01]
 [ 1.21388125e+00  6.23790324e-01 -5.96537292e-01  1.48945596e+00]
 [ 2.13137150e-01  4.65320349e-02  5.46879843e-02  2.24907649e-01]
 [ 1.93576783e-01  4.73378658e-01  1.34081754e-04  5.30131338e-01]
 [ 8.45537424e-01 -1.87271923e-01  2.55338639e-01  1.02903795e+00]
 [-5.56844138e-02  9.12002474e-02 -4.50917706e-03  1.06951259e-01]
 [-7.22816139e-02  9.26492587e-02  1.26241222e-01  5.22936262e-01]
 [ 1.81490630e-02  7.89812282e-02 -1.55177554e-02  8.25119605e-


numEvents: 531
num_FSPs_toData: 30
leaves.shape: (531, 30, 4)
SA_target.shape: (531, 30)
global_tag.shape: (531, 31)
leaves[0]: [[ 3.47773284e-01  1.40692875e-01  2.68519670e-01  6.75692627e-01]
 [-2.47066975e-01 -6.80908654e-03  2.52634853e-01  3.53430082e-01]
 [-1.18703857e-01  1.74011683e+00  1.76700902e+00  2.48282467e+00]
 [-1.54577857e-02  4.22994085e-02  1.46970570e-01  1.53715749e-01]
 [ 2.60628350e-02 -3.83722037e-03  5.03823999e-03  2.68212507e-02]
 [ 1.29558165e-02 -3.46922018e-02  7.23728836e-02  8.12972098e-02]
 [-3.28330547e-02 -6.77816495e-02  1.16204679e-01  1.38477034e-01]
 [-5.51543415e-01 -1.01239669e+00 -1.59332454e-01  1.17218340e+00]
 [ 6.11193292e-02 -3.06889415e-01  3.58554512e-01  4.75897072e-01]
 [ 3.18378992e-02 -4.90501337e-02  1.86622962e-01  1.95570186e-01]
 [ 1.34594426e-01  4.13679332e-02  3.51999015e-01  4.03992777e-01]
 [-3.35836560e-02  3.46974917e-02  1.27367169e-01  1.36213705e-01]
 [ 2.01130331e-01  9.35073942e-03  3.31429020e-02  2.04057095e-01]



numEvents: 133
num_FSPs_toData: 33
leaves.shape: (133, 33, 4)
SA_target.shape: (133, 33)
global_tag.shape: (133, 34)
leaves[0]: [[-0.03843503 -0.0059533   0.0434081   0.05828341]
 [-0.15408657 -0.02005767  0.41518071  0.46475789]
 [ 0.0086897   0.00827962  0.03126832  0.03349285]
 [-0.03769078  0.02275753  0.02807786  0.05221941]
 [-0.29716778  0.35670045  0.17005613  0.49443199]
 [-0.12047002 -0.03492063  0.2354954   0.2668156 ]
 [ 0.28343791 -0.23104343  0.21886274  0.65217788]
 [ 0.03144202  0.009189    0.03433166  0.04745209]
 [-0.03362871  0.02978667  0.17207307  0.17784059]
 [-0.01877401 -0.11855627  0.09627858  0.15387533]
 [-0.01369782  0.03802297  0.12742972  0.13368512]
 [ 1.49165606 -1.00340748 -0.44686291  1.85244463]
 [-0.12159103  0.2760368   0.29679963  0.65022126]
 [ 0.06528965 -0.11974391  0.11512836  0.17848216]
 [-0.05469584  0.05915455  0.15159062  0.22124761]
 [ 0.01997216  0.03158737 -0.03873985  0.05382773]
 [ 0.18819538 -0.15046364  0.03454325  0.24341332]
 [-0


numEvents: 24
num_FSPs_toData: 36
leaves.shape: (24, 36, 4)
SA_target.shape: (24, 36)
global_tag.shape: (24, 37)
leaves[0]: [[-8.69816542e-02 -3.21587324e-02  2.44801491e-01  5.58788630e-01]
 [ 6.41732588e-02  1.08689498e-02 -5.72385713e-02  8.66752287e-02]
 [-1.73322223e-02  9.44346115e-02  5.73285148e-02  1.78842816e-01]
 [ 2.97526002e-01 -1.81628540e-01  1.54905185e-01  4.06184881e-01]
 [ 6.00035749e-02  1.99320279e-02  1.14101715e-01  1.30448903e-01]
 [-1.22269385e-01 -1.57912716e-01 -3.22907925e-01  6.22794289e-01]
 [ 1.15372106e-01  5.51846400e-02 -1.31516591e-01  1.83446671e-01]
 [-6.39539659e-02 -1.95361320e-02  2.19463874e-02  7.03804954e-02]
 [ 4.76599187e-01  4.76059288e-01  1.16427529e+00  1.35232987e+00]
 [-5.20788729e-01  9.36289027e-04  4.72599193e-02  7.19146889e-01]
 [-4.85928431e-02  5.62342405e-02 -3.02728247e-02  8.02495989e-02]
 [ 3.17776650e-02 -2.49897931e-02 -2.15532258e-03  4.04840113e-02]
 [-3.47405560e-02 -1.23509660e-01  1.00039160e-02  5.10175060e-01]
 [-1

In [37]:
print("saving is done")
now = datetime.now()
print("time at end =", now)

saving is done
time at end = 2021-09-24 21:17:17.733698
